In [1]:
import time
import multiprocessing 
import numpy as np
from multiprocessing import Pool, Array, Process

In [2]:
import numpy as np
from multiprocessing.pool import ThreadPool
import itertools
from multiprocessing import sharedctypes

In [3]:
def multiprocessing_func(xLim, yLim, shared_array):
    tmp = np.ctypeslib.as_array(shared_array)
    
    underLimX, upperLimX = xLim
    underLimY, upperLimY = yLim
    for i in range(underLimX, upperLimX):
        for j in range(underLimY, upperLimY):
            tmp[i,j]= i
        
    def run(self, X):
        starttime = time.time()
        processes = []
        
        x_len, y_len = X.shape
        
        result = np.ctypeslib.as_ctypes(np.zeros((x_len, y_len)))
        shared_array = sharedctypes.RawArray(result._type_, result)
        
        cores = multiprocessing.cpu_count()

        batchsize =  int(x_len / cores)
        
        
        for underLim in range(0, x_len, batchsize):
            if underLim + batchsize > x_len:
                upperLim = x_len
            else:
                upperLim = underLim + batchsize
                
            p = multiprocessing.Process(target=multiprocessing_func, args=((underLim, upperLim),(0,y_len), shared_array))
            processes.append(p)
            p.start()
        
        for process in processes:
            process.join()
        
        print('That took {} seconds'.format(time.time() - starttime))
        return np.ctypeslib.as_array(shared_array)
    

In [22]:
def getNumeratorCPU2(m, n, S, z, dtype, cores=-1):
    if cores == -1:
        cores = multiprocessing.cpu_count()
        
    
    N = np.zeros([S + 1, m], dtype)
    N_old = N.copy()
    
    
    N = np.ctypeslib.as_ctypes(np.zeros((S + 1, m)))
    N_shared_array = sharedctypes.RawArray(N._type_, N)
    
    def multiprocessing_func(xLim, yLim, N_shared_array):
        tmp = np.ctypeslib.as_array(N_shared_array)
    
        underLimX, upperLimX = xLim
        underLimY, upperLimY = yLim

    
        for j in range(underLimY, upperLimY):
            for s in range(underLimX, upperLimX):
                if j >= m + 1 or s > S or j < 1:
                    pass
                elif s == 0 and int(j-1)==0:
                    tmp[s,j-1] =  1
                elif i < j:
                    tmp[s, j - 1] = 0
                elif j > 1 and z[i-1] <= s:
                    tmp[s,j-1] = N_old[s - z[i -1], j-2] + N_old[s,j-1]
                elif j > 1 and z[i-1] > s:
                    tmp[s,j-1] = N_old[s,j-1]
    
    starttime = time.time()
        
    x_len, y_len = S + 1, m + 1

    batchsize =  int(x_len / cores)
        
    for i in range(1,(m+n)+1): 
        processes = []
        for underLim in range(0, x_len, batchsize):
            if underLim + batchsize > x_len:
                upperLim = x_len
            else:
                upperLim = underLim + batchsize
                
            p = multiprocessing.Process(target=multiprocessing_func, args=((underLim, upperLim),(0,y_len), N_shared_array))
            processes.append(p)
            p.start()
        
        for process in processes:
            process.join()
        
        N_old = np.ctypeslib.as_array(N_shared_array).copy()
        

    return N_old[:,-1]

In [23]:
%time D = getNumeratorCPU2(m+1, n, S, [0] + list(z), np.float64, 3)

CPU times: user 255 ms, sys: 413 ms, total: 667 ms
Wall time: 42.3 s


In [24]:
D

array([0., 0., 0., ..., 0., 1., 2.])

In [ ]:
m+n

In [5]:
n =65
upperL = 400

np.random.seed(1)
A = np.asarray([np.random.randint(0,upperL,n) for _ in range(1)])
B = np.asarray([np.random.randint(0,upperL,n) for _ in range(1)])

x = list(A[0])
y = list(B[0])
m = len(x)
n = len(y)
z = x + y;z.sort()
S = sum(z[m:])
dtype = np.uint16

In [7]:
def getNumeratorCPU1(m, n, S, z, dtype):
    N = np.zeros([S + 1, m], dtype)
    N_old = N.copy()
    
    for i in range(1,(m+n)+1):
        for j in range(0, m +1):
            for s in range(0,S+1):
                
                if j >= m + 1 or s > S or j < 1:
                    pass
                elif s == 0 and int(j-1)==0:
                    N[s,j-1] =  1
                elif i < j:
                    N[s, j - 1] = 0
                elif j > 1 and z[i-1] <= s:
                    N[s,j-1] = N_old[s - z[i -1], j-2] + N_old[s,j-1]
                elif j > 1 and z[i-1] > s:
                    N[s,j-1] = N_old[s,j-1]
    
        N_old = N.copy()


        
    return N_old

In [8]:
%time D2 = getNumeratorCPU1(m+1, n, S, [0]+list(z), np.float64)

CPU times: user 2min 1s, sys: 0 ns, total: 2min 1s
Wall time: 2min 1s


In [9]:
np.all(D2 == D)

True

In [ ]:
D2[:,-1]

In [ ]:
class test:
    def __init__(self):
        pass
    
    def fun(self, av, bv):
        time.sleep(0.5)
        return (av, bv)

    def data_stream(self, a, b):
        for i, av in enumerate(a):
            for j, bv in enumerate(b):
                yield (i, j), (av, bv)

    def proxy(self, args):
        self.arr[args[0][0], args[0][1]] = self.fun(*args[1])[0]
        

    def run(self, a,b, cores=7):
        self.arr = np.zeros((len(a),len(b)))
        pool = ThreadPool(processes=cores)
        pool.map(self.proxy, self.data_stream(a, b))
    
        return self.arr
    
    

In [ ]:
%time test().run(a,b)

In [ ]:
def comp(a,b):
    arr = np.zeros((len(a),len(b)))
    for i, av in enumerate(a):
        for j, bv in enumerate(b):
            time.sleep(0.5)
            arr[i,j] = av
    return arr

In [ ]:
a = range(100, 400, 50)
b = range(100, 400, 50)

In [ ]:
%time comp(list(a),list(b))

In [ ]:
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
import time, pprint

def fun(av, bv):
    time.sleep(0.5)
    return (av, bv)

def data_stream(a, b):
    for i, av in enumerate(a):
        for j, bv in enumerate(b):
            yield (i, j), (av, bv)

def proxy(args):
    return args[0], fun(*args[1])



In [ ]:
def main1():
    a = range(100, 400, 100)
    b = range(100, 400, 100)
    Y = {}

    pool = Pool(processes=1)
    results = pool.map(proxy, data_stream(a, b))
    
    return results
    
    

In [ ]:
def main2():
    a = range(100, 400, 100)
    b = range(100, 400, 100)
    Y = {}

    return proxy(data_stream(a, b))
    

In [ ]:
%time main1()

In [ ]:
%time main2()

In [ ]:
list(data_stream(a, b))

In [ ]:
proxy(list(data_stream(a, b)))

In [ ]:
a = range(100, 400, 100)
b = range(100, 400, 100)
Y = {}

pool = Pool(processes=4)
results = pool.map(proxy, data_stream(a, b))
for k,v in results:
    Y[k] = v

pprint.pprint(Y)

In [ ]:
 mp.freeze_support()

In [ ]:
%time main(10)

In [ ]:
%time proxy(list(data_stream(a, b)))

In [ ]:
class test2:
    def __init__(self):
        pass
    
    def fun(self, av, bv):
        y = x*x
        time.sleep(2)
        return y
    
    def data_stream(self, a, b):
        for i, av in enumerate(a):
            for j, bv in enumerate(b):
                yield (i, j), (av, bv)
                
    def proxy(self, args):
        self.arr[args[0][0], args[0][1]] = self.fun(*args[1])
    
    def multiprocessing_func(x,s):
        print(s)
        y = x*x
        time.sleep(2)
        print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
        
    def run(self):
        starttime = time.time()
        processes = []
        for i in range(0,10):
            p = multiprocessing.Process(target=multiprocessing_func, args=(i,"hello"))
            processes.append(p)
            p.start()
        
        for process in processes:
            process.join()
        
        print('That took {} seconds'.format(time.time() - starttime))
    
    
    

In [ ]:
test2().run()

In [ ]:
import time
import multiprocessing 

def fun(self, av, bv):
    y = x*x
    time.sleep(2)
    return y

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x,s):
    print(s)
    y = x*x
    time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
if __name__ == '__main__':
    starttime = time.time()
    processes = []
    for i in range(0,10):
        p = multiprocessing.Process(target=multiprocessing_func, args=(i,))
        processes.append(p)
        p.start()
        
    for process in processes:
        process.join()
        
    print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
def fun(I, J):
    for i in range(I):
        for j in range(J):
            pass
    print("Finnished")

In [ ]:
n=10_000

In [ ]:
%time fun(n, n)

In [ ]:
import itertools

In [ ]:
window_idxs = [(i, j) for i, j in
               itertools.product(range(0, n),
                                 range(0, n))]

In [ ]:
len(window_idxs)

In [ ]:
import time
import multiprocessing 

def fun(self, av, bv):
    y = x*x
    time.sleep(2)
    return y

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x):
    y = x*x
    time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
if __name__ == '__main__':
    starttime = time.time()
    processes = []
    for i in range(0,10):
        multiprocessing_func(i)
       
        
    for process in processes:
        process.join()
        
    print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
class test:
    def __init__(self):
        pass
    
    def fun(self, av, bv):
        time.sleep(0.1)
        return (av, bv)

    def data_stream(self, a, b):
        for i, av in enumerate(a):
            for j, bv in enumerate(b):
                yield (i, j), (av, bv)

    def proxy(self, args):
        self.arr[args[0][0], args[0][1]] = self.fun(*args[1])[0]
        #print(args[0])
        #print(args[1])
        #return args[0], self.fun(*args[1])

    def run(self, a,b, cores=7):
        self.arr = np.zeros((len(a),len(b)))
        pool = ThreadPool(processes=cores)
        pool.map(self.proxy, self.data_stream(a, b))
    
        return self.arr
    
    